# Molecular Dynamics: Stability

Before continue, you should have:

- installed GROMACS
- put CHARMM36 force field in the correct location
- generated a backbone structure
- predicted the sequence of the generated structure
- modelled and added side chains to the structure
- added H atoms to the structure

In [ ]:
import os

os.environ['PATH'] = '/usr/local/gromacs/bin:' + os.environ['PATH']

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [ ]:
PROTEIN_CLS = "GFP"
PROTEIN_SID = "FM_4"
PROTEIN_PDB = Path(f"../data/{PROTEIN_CLS}/generated/BQSH/{PROTEIN_SID}.pdb")

# Create a new working directory!
WD = Path(f"wd-md1/{PROTEIN_CLS}/{PROTEIN_SID}")
WD.mkdir(parents=True, exist_ok=True)

# TOOLS: Change the path accordingly
GROMACS_PATH = "~/WS/ProtMatch/gromacs"

Cleaning the input structure

Strip out all the atoms that do not belong to the protein (e.i crystal waters, ligands, etc).

In [ ]:
PROTEIN_TMP = Path(f"{PROTEIN_PDB.name}_tmp.pdb")
!grep -v HETATM {PROTEIN_PDB} > {WD}/{PROTEIN_TMP}

PROTEIN_PDB = Path(PROTEIN_PDB.name)
!cd {WD} && grep -v CONECT {PROTEIN_TMP} > {PROTEIN_PDB}

!cd {WD} && rm -f {PROTEIN_TMP}
del PROTEIN_TMP

Move the protein to the origin.

In [ ]:
PROTEIN_TMP = Path(f"{PROTEIN_PDB.stem}.pdb")
!cd {WD} && vmd -dispdev text -eofexit -args < {GROMACS_PATH}/to_origin.tcl {PROTEIN_PDB} {PROTEIN_TMP}
PROTEIN_PDB = PROTEIN_TMP
del PROTEIN_TMP

## Setting Up the Simulation Box and Converting File Format

- Distance between the solute and the box: 15 nm
- Box type: octahedron
- Center molecule in box.

In [ ]:
PROTEIN_GRO = Path(f"{PROTEIN_PDB.stem}.gro")
!cd {WD} && gmx editconf -f {PROTEIN_PDB} -o {PROTEIN_GRO} -c -d 1.5 -bt octahedron

## Generating Topology with CHARMM36 Force Field

Generate the topology for the protein structure using the **charmm36** force field and the **TIP3P** water model.
The topology file defines the molecular structure, including atomic types, charges, and bonding information, which is essential for subsequent molecular dynamics simulations.

> (From GROMACS 2024 Manual)
> CHARMM (Chemistry at HARvard Macromolecular Mechanics) is a both a set of force fields and a software package for molecular dynamics (page 328) simulations and analysis. Includes united atom (CHARMM19) and all atom (CHARMM22, CHARMM27, CHARMM36) force fields (page 328). The CHARMM27 force field has been ported to GROMACS and is officially supported. CHARMM36 force field files can be obtained from the MacKerell lab website, which regularly produces up-to-date CHARMM force field files in GROMACS format.
> 
> For using CHARMM36 in GROMACS, please use the following settings in the mdp (page 483) file:
> ```
> constraints = h-bonds
> cutoff-scheme = Verlet
> vdwtype = cutoff
> vdw-modifier = force-switch
> rlist = 1.2
> rvdw = 1.2
> rvdw-switch = 1.0
> coulombtype = PME
> rcoulomb = 1.2
> DispCorr = no
> ```
> Note that dispersion correction should be applied in the case of lipid monolayers, but not bilayers.
> Please also note that the switching distance is a matter of some debate in lipid bilayer simulations, and it is dependent to some extent on the nature of the lipid. Some studies have found that an 0.8-1.0 nm switch is appropriate, others argue 0.8-1.2 nm is best, and yet others stand by 1.0-1.2 nm. The user is cautioned to thoroughly investigate the force field literature for their chosen lipid(s) before beginning a simulation!


In [ ]:
PROTEIN_TOP_GRO = Path(f"{PROTEIN_PDB.stem}_top.gro")
PROTEIN_TOP = Path(f"{PROTEIN_PDB.stem}.top")
FF_PATH = Path("../gromacs/charmm36-jul2022.ff")
!cp -r {FF_PATH} {WD}
# !cd {WD} && gmx pdb2gmx -f {PROTEIN_GRO} -o {PROTEIN_TOP_GRO} -p {PROTEIN_TOP} -water tip3p -ff charmm36-jul2022
# Any errors check https://manual.gromacs.org/2021.4/user-guide/run-time-errors.html.
# If a fatal error occurred due to H, this atom may not defined in the FF.
# Consider to remove this atom from PDB, or (not recommended) ignore hydrogen atoms with `-ignh`.
!cd {WD} && gmx pdb2gmx -f {PROTEIN_GRO} -o {PROTEIN_TOP_GRO} -p {PROTEIN_TOP} -water tip3p -ff charmm36-jul2022 -ignh

In [ ]:
!cd {WD} && gmx editconf -f {PROTEIN_TOP_GRO} -o output.pdb

## Running Energy Minimization

Create a `.tpr` file which is a portable binary run input file that contains all the necessary information for performing an energy minimization on the protein structure in a vacuum.
Ensure that the structure is stable and free from significant steric clashes or unrealistic geometries before proceeding to further simulations.

In [ ]:
MIN_SD_MDP = Path(GROMACS_PATH) / "min_sd.mdp"
VACUUM_TPR = Path(f"{PROTEIN_PDB.stem}_vacuum.tpr")
!cd {WD} && gmx grompp -v -f {MIN_SD_MDP} -c {PROTEIN_TOP_GRO} -p {PROTEIN_TOP} -o {VACUUM_TPR} -maxwarn 1

Perform an energy minimization on the protein structure in a vacuum.
This is to eliminate any steric clashes or inappropriate geometry in the molecular structure.
It adjusts the positions of atoms to find a local minimum in the potential energy landscape, leading to a more stable conformation.

In [ ]:
VACUUM_GRO = Path(f"{VACUUM_TPR.stem}.gro")
!cd {WD} && gmx mdrun -v -deffnm {VACUUM_TPR.stem} -c {VACUUM_GRO} -gpu_id 0

Analyse and visualize the energy minimization results.

In [ ]:
VACUUM_EDR = Path(f"{VACUUM_GRO.stem}.edr")
VACUUM_XVG = Path(f"{VACUUM_GRO.stem}.xvg")
!cd {WD} && echo "11" | gmx energy -f {VACUUM_EDR} -o {VACUUM_XVG} -xvg none

fig, ax = plt.subplots(figsize=(3, 3))
df = pd.read_csv(WD / VACUUM_XVG, sep='\s+', header=None, names=['step', 'energy'])
plt.plot(df["step"], df["energy"], color="black")
plt.xlabel("step")
plt.ylabel("energy (kJ/mol)")
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
plt.grid(False)
plt.show()

## Solvating the System

Surrounding the protein with water molecules to create a realistic biological environment for molecular dynamics simulations.

In [ ]:
WATER_GRO = Path(f"{PROTEIN_PDB.stem}_water.gro")
!cd {WD} && gmx solvate -cp {VACUUM_GRO} -cs spc216.gro -p {PROTEIN_TOP} -o {WATER_GRO}

Remove any water molecules that may have inadvertently entered the core of the protein (within 3 Ångströms of the protein) during the solvation process.
Internal water molecules within the protein core can interfere with the protein's structure and function.

In [ ]:
WATER_RM_GRO = Path(f"{PROTEIN_PDB.stem}_waterRM.gro")
DEL_H2O_TCL = Path(GROMACS_PATH) / "del_wat_inside.tcl"
!cd {WD} && vmd -dispdev text -eofexit -args < {DEL_H2O_TCL} {WATER_GRO} {PROTEIN_TOP} {WATER_RM_GRO}

You may need to correct formatting issues in the GRO file generated during the molecular dynamics simulation setup process.
See https://github.com/allison-group/structural-phylogenetics-bootstrap/blob/master/MD/GMX/fix_gro.py

## Energy Minimization After Adding Waters

In [ ]:
WATER_RM_TPR = Path(f"{WATER_RM_GRO.stem}.tpr")
!cd {WD} && gmx grompp -v -f {MIN_SD_MDP} -c {WATER_RM_GRO} -p {PROTEIN_TOP} -o {WATER_RM_TPR} -maxwarn 1

Adding ions to neutralize the system:

Add the appropriate number of positive (Na+) and negative (Cl-) ions to the solvated protein system. We aim to approximate physiological conditions and use therefore a NaCl concentration of 0.15 M.

In [ ]:
SOLVATED_GRO = Path(f"{WATER_RM_TPR.stem}_solvated.gro")
!cd {WD} && echo SOL | gmx genion -s {WATER_RM_TPR} -o {SOLVATED_GRO} -conc 0.15 -pname NA -nname CL -neutral -p {PROTEIN_TOP}

Minimize energy again.

In [ ]:
SOLVATED_TPR = Path(f"{SOLVATED_GRO.stem}.tpr")
!cd {WD} && gmx grompp -v -f {MIN_SD_MDP} -c {SOLVATED_GRO} -p {PROTEIN_TOP} -o {SOLVATED_TPR}

In [ ]:
!cd {WD} && gmx mdrun -v -deffnm {SOLVATED_TPR.stem} -c {SOLVATED_GRO} -gpu_id 0

In [ ]:
VACUUM_EDR = Path(f"{SOLVATED_GRO.stem}.edr")
VACUUM_XVG = Path(f"{SOLVATED_GRO.stem}.xvg")
!cd {WD} && echo "11" | gmx energy -f {VACUUM_EDR} -o {VACUUM_XVG} -xvg none

fig, ax = plt.subplots(figsize=(3, 3))
df = pd.read_csv(WD / VACUUM_XVG, sep='\s+', header=None, names=['step', 'energy'])
plt.plot(df["step"], df["energy"], color="black")
plt.xlabel("step")
plt.ylabel("energy (kJ/mol)")
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
plt.grid(False)
plt.show()

## Equilibration - Temperature

Perform NVT (constant Number of particles, Volume, and Temperature) heating on the solvated protein system, 
where the system is gradually heated to the target temperature under constant volume conditions.

The main goals are:
- to slowly increase the temperature of the system, allowing it to adapt without causing structural distortions or instabilities.
- to equilibrate the kinetic energy distribution among the particles, ensuring a stable temperature before proceeding to subsequent simulation steps.

In [ ]:
NVT_MDP = Path(GROMACS_PATH) / "nvt.mdp"
NVT_NN = f"{PROTEIN_PDB.stem}_NVT"
NVT_TPR = Path(f"{NVT_NN}.tpr")
!cd {WD} && gmx grompp -v -f {NVT_MDP} -c {SOLVATED_GRO} -p {PROTEIN_TOP} -r {SOLVATED_GRO} -o {NVT_TPR}
!cd {WD} && gmx mdrun -v -deffnm {NVT_NN} -s {NVT_TPR} -gpu_id 0

In [ ]:
NVT_EDR = Path(f"{NVT_NN}.edr")
NVT_XVG = Path(f"{NVT_NN}.xvg")
!cd {WD} && echo "Temperature" | gmx energy -f {NVT_EDR} -o {NVT_XVG} -xvg none -b 20

fig, ax = plt.subplots(figsize=(3, 3))
df = pd.read_csv(WD / NVT_XVG, sep='\s+', header=None, names=['time', 'temperature'])
plt.plot(df["time"], df["temperature"], color="black")
plt.axhline(y=300, color='black', linestyle='--', linewidth=0.8)
plt.xlabel("time (ps)")
plt.ylabel("temperature (K)")
plt.grid(False)
plt.show()

## Equilibration - Pressure

NPT (constant Number of particles, Pressure, and Temperature) equilibration allows the system to adjust its volume to achieve the target temperature and pressure.

In [ ]:
NPT_MDP = Path(GROMACS_PATH) / "npt.mdp"
NVT_GRO = Path(f"{NVT_NN}.gro")
NVT_CPT = Path(f"{NVT_NN}.cpt")
NPT_NN = f"{PROTEIN_PDB.stem}_NPT"
NPT_TPR = Path(f"{NPT_NN}.tpr")
!cd {WD} && gmx grompp -v -f {NPT_MDP} -c {NVT_GRO} -t {NVT_CPT} -p {PROTEIN_TOP} -o {NPT_TPR}
!cd {WD} && gmx mdrun -v -deffnm {NPT_NN} -s {NPT_TPR} -gpu_id 0

In [ ]:
NPT_EDR = Path(f"{NPT_NN}.edr")
NPT_XVG = Path(f"{NPT_NN}.xvg")
!cd {WD} && echo "Pressure" | gmx energy -f {NPT_EDR} -o {NPT_XVG} -xvg none

fig, ax = plt.subplots(figsize=(3, 3))
df = pd.read_csv(WD / NPT_XVG, sep='\s+', header=None, names=['time', 'pressure'])
df['pressure_avg'] = df['pressure'].rolling(window=5).mean()
plt.plot(df["time"], df["pressure"], color="black", label='10-ps avg')
plt.plot(df["time"], df["pressure_avg"], color="red", linestyle='-', label='50-ps avg')
plt.axhline(y=1, color='black', linestyle='--', linewidth=0.8)
plt.xlabel("time (ps)")
plt.ylabel("pressure (bar)")
plt.legend(loc="lower right")
plt.grid(False)
plt.show()

In [ ]:
NPT_EDR = Path(f"{NPT_NN}.edr")
NPT_XVG = Path(f"{NPT_NN}_dens.xvg")
!cd {WD} && echo "Density" | gmx energy -f {NPT_EDR} -o {NPT_XVG} -xvg none

fig, ax = plt.subplots(figsize=(3, 3))
df = pd.read_csv(WD / NPT_XVG, sep='\s+', header=None, names=['time', 'density'])
df['density_avg'] = df['density'].rolling(window=5).mean()
plt.plot(df["time"], df["density"], color="black", label='10-ps avg')
plt.plot(df["time"], df["density_avg"], color="red", linestyle='-', label='50-ps avg')
plt.xlabel("time (ps)")
plt.ylabel("density (kg/m$^{3}$)")
plt.ylim((990, 1060))
plt.legend(loc="lower right")
plt.grid(False)
plt.show()

## Production

Perform a production simulation under NPT conditions to gather dynamic data about the system.
The goal is to simulate the behavior of the solvated and neutralized protein in an environment that mimics physiological conditions.

This phase collects detailed molecular dynamics data on the behavior of the protein and its surrounding solvent molecules under conditions that closely mimic a natural, physiological environment.

Running the simulation under constant pressure and temperature ensures that the system's volume can adjust dynamically, providing a more realistic representation of biological conditions.

The data obtained from this production run can be used to analyze various aspects of the protein's behavior, such as conformational changes, stability, interactions with solvent molecules, and other dynamic properties.

In [ ]:
PROD_MDP = Path(GROMACS_PATH) / "prod.mdp"
NPT_GRO = Path(f"{NPT_NN}.gro")
NPT_CPT = Path(f"{NPT_NN}.cpt")
PROD_NN = f"{PROTEIN_PDB.stem}_PROD"
PROD_TPR = Path(f"{PROD_NN}.tpr")
!cd {WD} && gmx grompp -v -f {PROD_MDP} -c {NPT_GRO} -t {NPT_CPT} -p {PROTEIN_TOP} -o {PROD_TPR}
!cd {WD} && gmx mdrun -v -deffnm {PROD_NN} -s {PROD_TPR} -gpu_id 0

In [ ]:
!rm -rf {WD}/charmm36-nov2016.ff

Now, convert some trajectories to PDB for subsequent analysis (ie. phylogenetic tree confidence).

In [ ]:
!cd {WD} && gmx check -f {NPT_NN}.xtc

In [ ]:
MAX_TIME = 200  # You can get this from the previous command.
os.mkdir(WD / "trajectories")
for i, t in enumerate(range(0, MAX_TIME, 10)):
    !cd {WD} && echo "Protein" | gmx trjconv -s {PROD_TPR} -f {PROD_NN}.xtc -o trajectories/trj{i}.pdb -dump {t}

## Stability Evaluation

### RMSD

Calculate the Root Mean Square Deviation (RMSD) of a molecular dynamics trajectory. 
RMSD measures the average deviation of a set of atomic positions from a reference structure, which is the initial structure. 

Here, we only calculate RMSD for Backbone (group 4).

1. **Low RMSD Values (0.1 - 0.3 nm or 1 - 3 $\mathring{\text{A}}$)**: The protein structure is very stable.

2. **Moderate RMSD Values (0.3 - 0.5 nm or 3 - 5 $\mathring{\text{A}}$)**: The protein structure is reasonably stable with some flexibility.

3. **High RMSD Values (>0.5 nm or >5 $\mathring{\text{A}}$)**: The protein structure may be unstable.

Regardless of the absolute RMSD value, a key indicator of stability is how the RMSD evolves over time. 
A stable protein will show an initial rise in RMSD as it equilibrates and then plateau, indicating it has reached a stable conformation.
 Once the RMSD reaches a steady state and fluctuates within a narrow range, the protein can be considered stable. 
 The exact RMSD value at the plateau will depend on the protein and the simulation conditions.

In [ ]:
PROD_XTC = Path(f"{PROD_NN}.xtc")
RMSD_OUT_XVG = Path("RMSD.xvg")
!cd {WD} && echo "4 4" | gmx rms -s {PROD_TPR} -f {PROD_XTC} -o {RMSD_OUT_XVG} -tu ns

In [ ]:
def read_xvg(file_path, columns):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith(('@', '#')):  # Skip header lines
                data.append([float(x) for x in line.split()])
    return pd.DataFrame(data, columns=columns)

In [ ]:
rmsd_df_columns = ["Time (ns)", "RMSD (nm)"]
rmsd_df = read_xvg(WD / RMSD_OUT_XVG, columns=rmsd_df_columns)
plt.figure(figsize=(3, 3))
plt.plot(rmsd_df[rmsd_df_columns[0]], rmsd_df[rmsd_df_columns[1]], label='RMSD', color="black")
plt.plot(rmsd_df[rmsd_df_columns[0]],
         rmsd_df[rmsd_df_columns[1]].rolling(window=200).mean(),
         label='0.1-ns avg', color="red")
plt.xlabel(rmsd_df_columns[0])
plt.ylabel(rmsd_df_columns[1])
plt.title('RMSD')
plt.grid(False)
plt.legend()
plt.show()
print(f"{np.mean(rmsd_df[rmsd_df_columns[1]])} pm {np.std(rmsd_df[rmsd_df_columns[1]])}")

### RMSF

Calculate the Root Mean Square Fluctuation (RMSF) of the atoms or residues in a molecular dynamics (MD) simulation.

By calculating RMSF, wecan identify which parts of the protein (or other macromolecule) are more flexible or dynamic. Regions with high RMSF values are more flexible, whereas regions with low RMSF values are more rigid.

The formula for RMSF is:
$$
\text{RMSF}(i) = \sqrt{\frac{1}{N} \sum_{t=1}^{N} (r_i(t) - \langle r_i \rangle)^2}
$$

1. **Low RMSF Values**: Regions of the protein with low RMSF values are relatively stable and do not deviate much from their average positions. These stable regions often correspond to secondary structures like alpha-helices and beta-sheets, which maintain their conformation throughout the simulation.

2. **High RMSF Values**: High RMSF values indicate flexible or dynamic regions that move significantly during the simulation. These regions can correspond to loops, termini, or other flexible parts of the protein.

In [ ]:
RMSF_OUT_XVG = Path("RMSF.xvg")
!cd {WD} && echo "4" | gmx rmsf -s {PROD_TPR} -f {PROD_XTC} -o {RMSF_OUT_XVG} -res

In [ ]:
rmsf_df_columns = ["Residue", "RMSF (nm)"]
rmsf_df = read_xvg(WD / RMSF_OUT_XVG, columns=rmsf_df_columns)
plt.figure(figsize=(3, 3))
plt.plot(rmsf_df[rmsf_df_columns[0]], rmsf_df[rmsf_df_columns[1]], label='RMSF', color="black")
plt.axhline(y=np.mean(rmsf_df[rmsf_df_columns[1]]), color="red", label="avg", linestyle="--")
plt.xlabel(rmsf_df_columns[0])
plt.ylabel(rmsf_df_columns[1])
plt.legend()
plt.title('RMSF')
plt.grid(False)
plt.show()
print(f"{np.mean(rmsf_df[rmsf_df_columns[1]])} pm {np.std(rmsf_df[rmsf_df_columns[1]])}")

### Radius of Gyration, Rg

Calculate the radius of gyration of a molecule throughout the MD simulation. The radius of gyration is a measure of the distribution of the mass of the molecule around its center of mass. A stable Rg value indicates that the protein is maintaining its overall shape and not unfolding or collapsing.

In [ ]:
RG_OUT_XVG = Path("RG.xvg")
!cd {WD} && echo "4" | gmx gyrate -s {PROD_TPR} -f {PROD_XTC} -o {RG_OUT_XVG}

In [ ]:
rg_df_columns = ["Time (ps)", "$R_g$", "$R_g$/sX/N", "$R_g$/sY/N", "$R_g$/sZ/N"]
rg_df = read_xvg(WD / RG_OUT_XVG, columns=rg_df_columns)
plt.figure(figsize=(3, 3))
for i in range(1, len(rg_df_columns)):
    plt.plot(rg_df[rg_df_columns[0]], rg_df[rg_df_columns[i]], label=rg_df_columns[i])
    print(f"{np.mean(rg_df[rg_df_columns[i]])} pm {np.std(rg_df[rg_df_columns[i]])}")
    # plt.plot(rg_df[rg_df_columns[0]], 
    #          rg_df[rg_df_columns[i]].rolling(window=200).mean())
plt.xlabel('Time (ps)')
plt.ylabel('Radius (nm)')
plt.title('$R_g$')
# plt.legend(loc='right', bbox_to_anchor=(1.52, 0.5))
plt.legend(loc='lower right')
plt.grid(False)
# plt.ylim((0, 1.5))
plt.show()

### Secondary Structure Count

Track changes in its secondary structure elements, such as alpha-helices and beta-sheets. How the overall secondary structure composition of the protein changes during the simulation?
   
If the number of residues in helices, sheets, or other structures remains relatively constant, it indicates structural stability.

In [ ]:
DSSP_OUT_XVG = Path("DSSP.xvg")
!cd {WD} && gmx dssp -s {PROD_TPR} -f {PROD_XTC} -o {DSSP_OUT_XVG.stem}.dat -num {DSSP_OUT_XVG}

In [ ]:
dssp_df_columns = ["Time (ps)", "Loops", "Breaks", "Bends", "Turns", "PP_Helices", "$\\pi$-Helices", "$3_{10}$-Helices",
                   "$\\beta$-Strands", "$\\beta$-Bridges", "$\\alpha$-Helices"]
dssp_df = read_xvg(WD / DSSP_OUT_XVG, columns=dssp_df_columns)
plt.figure(figsize=(3, 3))
for i in range(1, len(dssp_df_columns)):
    plt.plot(dssp_df[dssp_df_columns[0]], dssp_df[dssp_df_columns[i]], label=dssp_df_columns[i])
    print(f"{np.mean(dssp_df[dssp_df_columns[i]])} pm {np.std(dssp_df[dssp_df_columns[i]])}")
plt.xlabel('Time (ps)')
plt.ylabel('Secondary Structures Count')
plt.title('Secondary Structure')
plt.legend(loc='right', bbox_to_anchor=(1.6, 0.5))
plt.grid(False)
plt.show()

### Energy Analysis

In [ ]:
PROD_EDR = Path(f"{PROD_NN}.edr")

In [ ]:
TOTAL_ENERGY_XVG = Path(f"{PROD_NN}_total_energy.xvg")
!cd {WD} && echo "Total-Energy" | gmx energy -f {PROD_EDR} -o {TOTAL_ENERGY_XVG} -xvg none

POTENTIAL_ENERGY_XVG = Path(f"{PROD_NN}_potential_energy.xvg")
!cd {WD} && echo "Potential" | gmx energy -f {PROD_EDR} -o {POTENTIAL_ENERGY_XVG} -xvg none

KINETIC_ENERGY_XVG = Path(f"{PROD_NN}_kinetic_energy.xvg")
!cd {WD} && echo "Kinetic-En." | gmx energy -f {PROD_EDR} -o {KINETIC_ENERGY_XVG} -xvg none

In [ ]:
tot_en_df = pd.read_csv(WD / TOTAL_ENERGY_XVG, sep='\s+', header=None, names=['step', 'Total-Energy'])
pot_en_df = pd.read_csv(WD / POTENTIAL_ENERGY_XVG, sep='\s+', header=None, names=['step', 'Potential-Energy'])
ktc_en_df = pd.read_csv(WD / KINETIC_ENERGY_XVG, sep='\s+', header=None, names=['step', 'Kinetic-Energy'])


print(f"{np.mean(tot_en_df['Total-Energy'])} pm {np.std(tot_en_df['Total-Energy'])}")
print(f"{np.mean(pot_en_df['Potential-Energy'])} pm {np.std(pot_en_df['Potential-Energy'])}")
print(f"{np.mean(ktc_en_df['Kinetic-Energy'])} pm {np.std(ktc_en_df['Kinetic-Energy'])}")

fig, ax = plt.subplots(figsize=(3, 3))
plt.plot(tot_en_df["step"], tot_en_df["Total-Energy"], label="total energy")
plt.plot(pot_en_df["step"], pot_en_df["Potential-Energy"], label="potential energy")
plt.plot(ktc_en_df["step"], ktc_en_df["Kinetic-Energy"], label="kinetic energy")
plt.xlabel("time (ps)")
plt.ylabel("energy (kJ/mol)")
plt.legend()
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
plt.grid(False)
plt.title("Energy")
plt.show()

### Contact Map (Deperated)

A contact map is a 2D matrix where each cell (i, j) represents the contact (usually within a certain distance threshold, like 0.5 nm) between residue i and residue j over the course of the simulation. A high value in a cell indicates frequent contacts between those residues.

**Consistency of Contacts**:
- **Stable Regions**: Residues that maintain consistent contacts throughout the simulation indicate stable regions of the protein. In the contact map, these appear as persistent spots or clusters of high values.
- **Unstable Regions**: Regions where contacts are inconsistent or transient indicate flexible or unstable parts of the protein. These regions will show as dispersed or low-value spots in the contact map.

In [ ]:
# CONTACT_XVG = Path("CONTACT.xvg")
# CONTACT_XPM = Path(f"{CONTACT_XVG.stem}.xpm")
# !cd {WD} && echo "4" | gmx mdmat -s {PROD_TPR} -f {PROD_XTC} -mean {CONTACT_XPM} -no {CONTACT_XVG}

In [ ]:
# !cd {WD} && gmx xpm2ps -f {CONTACT_XPM} -o {CONTACT_XPM.stem}.eps
# import matplotlib.image as mpimg
# 
# img = mpimg.imread(WD / f"{CONTACT_XPM.stem}.eps")
# 
# plt.figure(figsize=(2, 2), dpi=300)
# plt.imshow(img)
# plt.axis('off')
# plt.show()